In [1]:
import fasttext

In [4]:
!wget https://dl.fbaipublicfiles.com/fasttext/data/cooking.stackexchange.tar.gz && tar xvzf cooking.stackexchange.tar.gz

--2020-08-17 16:00:56--  https://dl.fbaipublicfiles.com/fasttext/data/cooking.stackexchange.tar.gz
Resolving dl.fbaipublicfiles.com... 104.22.75.142, 172.67.9.4, 104.22.74.142
Connecting to dl.fbaipublicfiles.com|104.22.75.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 457609 (447K) [application/x-tar]
Saving to: ‘cooking.stackexchange.tar.gz’

cooking.stackexchan 100%[===================>] 446.88K   550KB/s    in 0.8s    

2020-08-17 16:00:58 (550 KB/s) - ‘cooking.stackexchange.tar.gz’ saved [457609/457609]

x cooking.stackexchange.id
x cooking.stackexchange.txt
x readme.txt


In [5]:
!head cooking.stackexchange.txt

__label__sauce __label__cheese How much does potato starch affect a cheese sauce recipe?
__label__food-safety __label__acidity Dangerous pathogens capable of growing in acidic environments
__label__cast-iron __label__stove How do I cover up the white spots on my cast iron stove?
__label__restaurant Michelin Three Star Restaurant; but if the chef is not there
__label__knife-skills __label__dicing Without knife skills, how can I quickly and accurately dice vegetables?
__label__storage-method __label__equipment __label__bread What's the purpose of a bread box?
__label__baking __label__food-safety __label__substitutions __label__peanuts how to seperate peanut oil from roasted peanuts at home?
__label__chocolate American equivalent for British chocolate terms
__label__baking __label__oven __label__convection Fan bake vs bake
__label__sauce __label__storage-lifetime __label__acidity __label__mayonnaise Regulation and balancing of readymade packed mayonnaise and other sauces


In [7]:
!wc cooking.stackexchange.txt

   15404  169582 1401900 cooking.stackexchange.txt


In [8]:
!head -n 12404 cooking.stackexchange.txt > cooking.train

In [9]:
!tail -n 3000 cooking.stackexchange.txt > cooking.valid

In [12]:
import fasttext
model = fasttext.train_supervised(input="cooking.train")

In [16]:
model.save_model("model_cooking.bin")

In [22]:
model.predict("Why not put knives in the dishwasher?", k=5)

(('__label__baking',
  '__label__food-safety',
  '__label__equipment',
  '__label__bread',
  '__label__substitutions'),
 array([0.06218182, 0.06068703, 0.03810307, 0.03686156, 0.03323049]))

In [23]:
model.predict("Which baking dish is best to bake a banana bread ?", k=5)

(('__label__baking',
  '__label__bread',
  '__label__food-safety',
  '__label__equipment',
  '__label__substitutions'),
 array([0.06488238, 0.04015051, 0.03951197, 0.03398247, 0.03265003]))

In [20]:
model.test("cooking.valid")

(3000, 0.15533333333333332, 0.06717601268559896)

In [21]:
model.test("cooking.valid", k=5)

(3000, 0.06626666666666667, 0.1432896064581231)

### Making the model better

In [24]:
cat cooking.stackexchange.txt | sed -e "s/\([.\!?,'/()]\)/ \1 /g" | tr "[:upper:]" "[:lower:]" > cooking.preprocessed.txt

In [26]:
!head -n 12404 cooking.preprocessed.txt > cooking.train

In [27]:
!tail -n 3000 cooking.preprocessed.txt > cooking.valid

In [28]:
model = fasttext.train_supervised(input="cooking.train")

In [29]:
model.test("cooking.valid")

(3000, 0.16933333333333334, 0.07323050309932247)

In [30]:
!head cooking.train

__label__sauce __label__cheese how much does potato starch affect a cheese sauce recipe ? 
__label__food-safety __label__acidity dangerous pathogens capable of growing in acidic environments
__label__cast-iron __label__stove how do i cover up the white spots on my cast iron stove ? 
__label__restaurant michelin three star restaurant; but if the chef is not there
__label__knife-skills __label__dicing without knife skills ,  how can i quickly and accurately dice vegetables ? 
__label__storage-method __label__equipment __label__bread what ' s the purpose of a bread box ? 
__label__baking __label__food-safety __label__substitutions __label__peanuts how to seperate peanut oil from roasted peanuts at home ? 
__label__chocolate american equivalent for british chocolate terms
__label__baking __label__oven __label__convection fan bake vs bake
__label__sauce __label__storage-lifetime __label__acidity __label__mayonnaise regulation and balancing of readymade packed mayonnaise and other sauces


In [31]:
model = fasttext.train_supervised(input="cooking.train", epoch=25)

In [35]:
model.test("cooking.valid")

(3000, 0.5156666666666667, 0.22300706357214933)

In [36]:
model = fasttext.train_supervised(input="cooking.train", epoch=25, lr=1.0)

In [37]:
model.test("cooking.valid")

(3000, 0.5836666666666667, 0.2524145884388064)

In [38]:
model = fasttext.train_supervised(input="cooking.train", lr=1.0, epoch=25, wordNgrams=2)

In [39]:
model.test("cooking.valid")

(3000, 0.607, 0.26250540579501225)

In [40]:
model = fasttext.train_supervised(input="cooking.train", lr=1.0, epoch=25, wordNgrams=2, bucket=200000, dim=50, loss='hs')

### Multi-label classification

In [41]:
model = fasttext.train_supervised(input="cooking.train", lr=0.5, epoch=25, wordNgrams=2, bucket=200000, dim=50, loss='ova')

In [42]:
model.predict("Which baking dish is best to bake a banana bread ?", k=-1, threshold=0.5)

(('__label__baking',
  '__label__bread',
  '__label__equipment',
  '__label__bananas'),
 array([1.00001001, 0.98968184, 0.97904575, 0.81287736]))

In [45]:
model.test("cooking.valid", k=-1)

(3000, 0.003146031746031746, 1.0)

In [44]:
model.predict("Which baking dish is best to bake a banana bread ?", k=-1, threshold=0.5)

(('__label__baking',
  '__label__bread',
  '__label__equipment',
  '__label__bananas'),
 array([1.00001001, 0.98968184, 0.97904575, 0.81287736]))